In [ ]:
!python segment/predict.py --weights weights/segment/yolov5s-seg-idcard-best-2.pt --max-det 1 --source data/images/autogen/cccdc_front_2 --save-txt


In [1]:
import torch
import utils

In [4]:
import os
import glob
import cv2
import numpy as np
from pathlib import Path
import math
import pandas as pd  # Import pandas for working with Excel files
image_dir = 'data/images/autogen/cccdc_front_2'
text_dir = 'runs/predict-seg/exp11/labels'
output_dir = './output'
# Define the Excel output file path
excel_output_file = './output/log.xlsx'
os.makedirs(output_dir, exist_ok=True)

# Create an empty DataFrame to store image names and angles
data = {
    'Image Name': [],
    'Angle': []
}
df = pd.DataFrame(data)

for image_path in glob.glob(os.path.join(image_dir, '*.jpg')) + glob.glob(os.path.join(image_dir, '*.JPEG')):
    image_name = os.path.basename(image_path)
    text_name = os.path.splitext(image_name)[0] + '.txt'
    text_path = os.path.join(text_dir, text_name)
    if os.path.exists(text_path):
        # Doc anh
        image = cv2.imread(image_path)
        h, w = image.shape[:2]
        # Doc txt

        with open(text_path, 'r') as file:
            lines = file.readlines()
        lines = [line for line in lines if len(line.strip().split()) > 1]
        text = '\n'.join(lines)
        values = lines[0].split()[1:]
        float_values = [float(v) for v in values]
        points = [(w * float_values[i], h * float_values[i + 1]) for i in range(0, len(float_values), 2)]

        # Cat segment
        mask = np.zeros_like(image[:, :, 0])
        points = np.array(points, np.int32)
        # Tim min box
        rect = cv2.minAreaRect(points)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.fillPoly(mask, [box], (255, 255, 255))
        result = cv2.bitwise_and(image, image, mask=mask)
        image = np.zeros((100, 100, 3), np.uint8)


    #     cv2.polylines(result, [points], isClosed=True, color=(0, 255, 0), thickness=2)
    #     cv2.drawContours(result, [box], 0, (0, 0, 255), 6)
    #     cv2.imshow("Bbox", result)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()
        rect = cv2.minAreaRect(points)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        # Calculate distances between all pairs of points
        point1 = box[0]
        if np.linalg.norm(box[0] - box[1]) > np.linalg.norm(box[0] - box[3]):
            point2 = box[1]
        else:
            point2 = box[3]
        x1, y1 = point1
        x2, y2 = point2
        if x1!=x2:
            angle = math.degrees(math.atan((y2-y1)/(x2-x1)))
        else:
            angle = -90

        # xoay anh
        #h, w = result.shape[:2]
        new_w = int(w * abs(np.cos(np.radians(angle))) + h * abs(np.sin(np.radians(angle))))
        new_h = int(h * abs(np.cos(np.radians(angle))) + w * abs(np.sin(np.radians(angle))))
        d = max(w, h)
        center = (d//2, d//2)

        M = cv2.getRotationMatrix2D(center, angle, 1)
        rotated_image = cv2.warpAffine(result, M, (d, d))

        # Save
        output_path = os.path.join(output_dir, image_name)
        output_path = Path(output_path)
        if output_path.suffix.lower() not in ['.jpg', '.jpeg']:
            output_path = output_path.with_suffix('.jpg')
        cv2.imwrite(str(output_path), rotated_image)
        # Append image name and angle to the DataFrame
        df = pd.concat([df, pd.DataFrame({'Image Name': [os.path.splitext(image_name)[0]], 'Angle': [angle]})], ignore_index=True)


# Ensure the directory exists before attempting to save the Excel file
output_dir_excel = os.path.dirname(excel_output_file)
if not os.path.exists(output_dir_excel):
    os.makedirs(output_dir_excel)

# Check if the Excel file exists, and if not, create it
if not os.path.exists(excel_output_file):
    df.to_excel(excel_output_file, index=False, sheet_name='Sheet1')  # Create a new Excel file with headers
else:
    # If the file exists, open it and append data to the existing sheet
    with pd.ExcelWriter(excel_output_file, engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, index=False, header=False)  # Append data without headers

C:\Users\Admin\AppData\Local\Temp\ipykernel_22584\303119955.py:46: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22584\303119955.py:59: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
